In [1]:
import sys
sys.path.insert(1,'./Instances_decoders')
sys.path.insert(1,'./Instances_models')
sys.path.insert(1,'./Evaluators')
from TSCFLP_greedy_decoder import greedy_decoder
from TSCFLP_instance import TSCFLPinstance
from evaluators import evaluator_Best_Mean
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

In [2]:
class Generator(nn.Module):
    def __init__(self,I,J,population):
        super().__init__()
        self.population = population
        self.chromosome_size = I+J
        self.rede =nn.Sequential(
            nn.Linear(self.chromosome_size,self.chromosome_size),
            nn.ReLU(),
            nn.Linear(self.chromosome_size,self.chromosome_size*self.population),
            nn.Sigmoid()
        )
    def forward(self,ruido):
        x = self.rede(ruido)
        x = torch.reshape(x,(self.population,150))
        return x
        
        

In [3]:
model = Generator(50,100,200)
instancia = TSCFLPinstance(r"C:\Users\Felipe\Projetos\TSCFLP\Instances\PSC1-C1-50.txt")
decoder = greedy_decoder(instancia)
evaluator = evaluator_Best_Mean(decoder)
lr = 0.1
optimizer = torch.optim.SGD(model.parameters(),lr=lr)
Epochs = 500

In [8]:
def train_loop(model,evaluator,optimizer,Epochs):
    epochs_loss = []
    
    
    for epoch in range(Epochs):
        ruido = torch.randn(150)
        population = model(ruido)
        
  
        optimizer.zero_grad()
 
        loss = evaluator(population)
        loss.requires_grad_()
        
        loss.backward()
        
        optimizer.step()
        epochs_loss.append(loss.item())
        
        if epoch%10==0:
            print(f'Melhor individuo:{evaluator.best_fit} Média da população: {evaluator.mean_current_pop}')
    return epochs_loss,evaluator

In [12]:
ruido = torch.randn(150)
population = model(ruido)
t = evaluator.best_chromossome(population)
print(t)

[(1009248, 122), (1008598, 110), (1004252, 156), (1003397, 160), (1000316, 149), (999893, 94), (998437, 162), (998212, 50), (998114, 61), (996666, 78), (994848, 152), (992159, 167), (991415, 86), (989994, 130), (989970, 150), (989838, 165), (989800, 146), (988832, 179), (987826, 25), (984485, 79), (984395, 195), (983860, 123), (982874, 85), (982441, 66), (981287, 116), (981085, 142), (980821, 52), (980244, 157), (979830, 74), (978525, 51), (977878, 176), (976746, 48), (975828, 98), (975578, 141), (974439, 131), (974416, 173), (974087, 42), (973196, 164), (971964, 9), (971769, 168), (970619, 175), (970564, 181), (970451, 158), (969859, 137), (968263, 153), (967589, 139), (966725, 99), (966663, 124), (966406, 63), (966378, 10), (965164, 114), (964669, 132), (963556, 58), (962846, 69), (961988, 126), (961907, 147), (961875, 148), (961855, 91), (961784, 95), (961655, 182), (960982, 109), (959852, 70), (959740, 194), (959177, 6), (959033, 199), (959023, 46), (957666, 72), (956973, 14), (956

In [14]:
best = population[174]

In [17]:
print(best)

tensor([0.4032, 0.5179, 0.5873, 0.4831, 0.5481, 0.5190, 0.5950, 0.5136, 0.4594,
        0.4951, 0.5486, 0.4288, 0.4777, 0.4564, 0.4125, 0.4995, 0.4633, 0.4884,
        0.4805, 0.4027, 0.5687, 0.6138, 0.4690, 0.4392, 0.4201, 0.3802, 0.5458,
        0.4720, 0.5250, 0.5819, 0.4902, 0.5111, 0.4916, 0.5870, 0.4884, 0.4161,
        0.4912, 0.6003, 0.4856, 0.4958, 0.4574, 0.4806, 0.4618, 0.4083, 0.4341,
        0.4541, 0.4763, 0.4662, 0.5222, 0.4621, 0.4330, 0.4434, 0.4915, 0.4482,
        0.4982, 0.5397, 0.5087, 0.4716, 0.5785, 0.4874, 0.5281, 0.4700, 0.5306,
        0.5359, 0.5364, 0.4889, 0.5644, 0.4544, 0.4446, 0.4392, 0.4586, 0.3995,
        0.3644, 0.5124, 0.5161, 0.4517, 0.4885, 0.5038, 0.4254, 0.4992, 0.4542,
        0.4408, 0.3745, 0.4168, 0.5570, 0.6625, 0.6119, 0.6121, 0.4344, 0.5319,
        0.5116, 0.5744, 0.5124, 0.4472, 0.4477, 0.5078, 0.5652, 0.5163, 0.4494,
        0.4196, 0.5265, 0.5105, 0.5988, 0.5914, 0.4888, 0.4149, 0.4737, 0.4145,
        0.4609, 0.4086, 0.4873, 0.5368, 

In [20]:
loss = []
func = torch.nn.MSELoss()
for i in range(len(population)):
    loss.append(func(best,population[i]))

In [22]:
print(len(population))

200


In [23]:
print(loss[174])

tensor(0., grad_fn=<MseLossBackward0>)


In [ ]:
import matplotlib.pyplot as plt
plt.plot(epoch_loss)

In [ ]:
plt.plot(evaluator.epochs_mean_fit)